In [4]:
import bibtexparser as bp
import pandas as pd
import numpy as np
import yaml as yml

# CARREGA O ARQUIVO DE CONFIGURAÇÃO
dirConfig = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos'
configFileName = 'config_analisa_bibtex.yml'

with open(dirConfig + '\\' + configFileName) as f:
    configFile = yml.load(f, Loader=yml.loader.SafeLoader)


In [5]:
# ANALISA BIBTEX
listaArquivos = ( \
	configFile['FILE_ACM'], \
	configFile['FILE_IEE'], \
	configFile['FILE_SD' ]
	)

formatFile = configFile['FORMATO']

dirInput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\input'
dirOutput = r'D:\Impacta\Python for Data Engineer\Aula 2\Arquivos\output'

mergedFilesCSV = 'ALL_ARTICLES.csv'
mergedFilesJSON = 'ALL_ARTICLES.json'
mergedFilesYAML = 'ALL_ARTICLES.yml'

def bibtexToDict(arqv):
	arquivo = arqv
	with open(dirInput + '\\' + arquivo, encoding='utf8') as f:
		bib_database = bp.load(f)		
	return bib_database.entries_dict.values()
	
# UNION DATAFRAMES AND YML
listaDf = []
listaYml =[]
for i in listaArquivos:
	sourceArticles = bibtexToDict(i)
	listaDf.append(pd.DataFrame(sourceArticles))
	listaYml.append(yml.dump(list(sourceArticles)))

unionDf = pd.concat(listaDf)
unionYml = ''.join(listaYml)

In [6]:
# EXPORT
if formatFile == 'CSV':
	unionDf.to_csv(dirOutput + '\\' + mergedFilesCSV, index=False, encoding='utf-8')
elif formatFile == 'JSON':
	unionDf.to_json(dirOutput + '\\' + mergedFilesJSON, orient = 'records')
elif formatFile == 'YAML':
	with open(dirOutput + '\\' + mergedFilesYAML, mode='w',encoding='utf8') as f:
		f.write(unionYml)
else:
	print('Formato não disponivel.')
